# Prepping Data

In [1]:
from datasets import load_dataset

In [3]:
from datasets import Dataset, load_dataset
import pandas as pd
df = pd.read_csv('/notebooks/data/artist_catalogs/CSV/curated_artistdb.csv')

#, nrows=1000

In [4]:
df

,artist,song_title,lyrics
0,Immortal_Technique,Angels & Demons,What do you see when you're in the dark and th...
1,Immortal_Technique,Anomalies,Ayyo\nYou’re blind to the fact ‘cause you’re a...
2,Immortal_Technique,Beef and Broccoli,"Look, let me make something abundantly clear f..."
3,Immortal_Technique,Belly of the Beast,"This hip-hop verse is somewhat of a change, so..."
4,Immortal_Technique,Bin Laden,"Man, you hear this bullshit they be talking\nE..."
...,...,...,...
10416,Army_of_the_Pharaohs,War Ensemble,"""In a war there are many moments for compassio..."
10417,Army_of_the_Pharaohs,War Machine,Mutilate massacre mummies that's comin' to mai...
10418,Army_of_the_Pharaohs,Warth of Gods,I could talk bitches out of they jeans\nGold d...
10419,Army_of_the_Pharaohs,Wrath of Gods,I could talk bitches out of they jeans\nGold d...


In [5]:
#targeting the key variables for training
df_lyrics = df[['artist','song_title','lyrics']]

In [7]:
#cleaning dataset where there are no lyrics in a dataset
df_lyrics= df_lyrics[df_lyrics['lyrics']!=None]

In [8]:
#review how many rows have been deleted
df_lyrics.describe()

,artist,song_title,lyrics
count,10421,10419,10421
unique,79,9900,10260
top,Nas,Intro,Girl you know that you need\nTo stop givin' me...
freq,429,26,12


In [9]:
#dropping duplicate rows
df_lyrics= df_lyrics[df_lyrics['lyrics']!=None].drop_duplicates(subset=['lyrics'])

In [10]:
#dropping rows with null values
df_lyrics = df_lyrics.dropna()

In [11]:
#checking to see how many rows have been deleted after cleaning
df_lyrics.describe()

,artist,song_title,lyrics
count,10258,10258,10258
unique,78,9781,10258
top,Nas,Intro,What do you see when you're in the dark and th...
freq,427,26,1


In [12]:
df_lyrics.iloc[564]

artist                                                      Nas
song_title                               Eye 4 An Eye Freestyle
lyrics        Yo this Nas niggas whuttup?\nQB Album coming, ...
Name: 564, dtype: object

In [13]:
df_lyrics["lyrics"][564]

"Yo this Nas niggas whuttup?\nQB Album coming, niggas Ill Will, Bravehearts\nMy nigga DJ Clue - put this shit on these niggas\nDesert Storm baby 2000!\nA Drug dealers dream - flip his last Ki out the game\nCause entertainment money now is off the chain\nI put blunts in niggas caskets, lost in the game\nI learned about pain, bodies burnt in drug game\nBut I'm neva gon' die, I neva heard of death\nEnergy could neva be destroyed\nOnly the flesh, when you niggas try to murder me\nWith bullets to my head\nThis is my you can't kill me niggas\nI'm already dead, BRING IT!!\nI'm busting bodies fall in the cemetery\nBlack blood raining on the street\nAll you niggas, buried y'all\nNiggas just came home, fresh out the state greens\nFive time felon, thinking he jelling in Queens\nIt's pitiful - what bid could do, especially federal\nY'all young thugs wilding, see prison\nGot a bed for you waiting, I clapped that Satan\nBut thought I was dreaming\nI woke up masturbating in the bed with demons\nI cri

In [14]:
#Source - https://huggingface.co/docs/datasets/loading#pandas-dataframe
#Source - https://huggingface.co/docs/datasets/v2.9.0/en/package_reference/loading_methods#datasets.load_dataset.split
#maybe we need to split the data source during loading
dataset = Dataset.from_pandas(df_lyrics)

In [15]:
dataset.features

{'artist': Value(dtype='string', id=None),
 'song_title': Value(dtype='string', id=None),
 'lyrics': Value(dtype='string', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [16]:
dataset = dataset.remove_columns(["song_title"])
dataset.features

{'artist': Value(dtype='string', id=None),
 'lyrics': Value(dtype='string', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [17]:
dataset = dataset.remove_columns(["artist"])
dataset.features

{'lyrics': Value(dtype='string', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [18]:
dataset = dataset.remove_columns(["__index_level_0__"])
dataset.features

{'lyrics': Value(dtype='string', id=None)}

# split dataset

In [18]:
#split the data into train and test set
#from sklearn.model_selection import train_test_split

#ds_train, ds_test = train_test_split(dataset, test_size=0.2, random_state=42, shuffle=True)

# Build Transformers

In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorWithPadding

context_length = 500
checkpoint = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

outputs = tokenizer(
    dataset["lyrics"][:2],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

Input IDs length: 4
Input chunk lengths: [500, 197, 500, 269]
Chunk mapping: [0, 0, 1, 1]


# tokenize def

In [20]:
def tokenize(element):
    outputs = tokenizer(
        element["lyrics"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = dataset.map(
    tokenize, batched=True, remove_columns=["lyrics"])
tokenized_datasets

  0%|          | 0/11 [00:00<?, ?ba/s]

Dataset({
    features: ['input_ids'],
    num_rows: 10962
})

In [21]:
tokenized_datasets[564]

{'input_ids': [3855,
  12431,
  44873,
  4908,
  198,
  6,
  42323,
  314,
  1101,
  64,
  651,
  6164,
  198,
  818,
  2233,
  640,
  198,
  28172,
  17753,
  18340,
  198,
  3855,
  12431,
  11,
  314,
  11662,
  299,
  1127,
  198,
  40,
  910,
  428,
  530,
  640,
  198,
  464,
  938,
  640,
  11,
  314,
  21192,
  284,
  331,
  6,
  439,
  198,
  40,
  1101,
  422,
  810,
  198,
  464,
  45654,
  25912,
  2412,
  318,
  379,
  198,
  1870,
  25542,
  27318,
  198,
  1870,
  3013,
  9249,
  869,
  1644,
  198,
  2202,
  777,
  299,
  6950,
  292,
  11,
  290,
  4675,
  198,
  41389,
  364,
  7288,
  338,
  588,
  198,
  7556,
  36749,
  6,
  5727,
  290,
  198,
  16501,
  1648,
  259,
  6,
  299,
  6950,
  292,
  287,
  6698,
  290,
  923,
  259,
  6,
  10512,
  290,
  198,
  2484,
  1025,
  259,
  6,
  299,
  6950,
  292,
  11,
  4692,
  2910,
  11,
  655,
  1494,
  705,
  368,
  11,
  523,
  644,
  198,
  3152,
  262,
  11511,
  6,
  14896,
  467,
  14380,
  198,
  28971,
  3105,

# initialized model

In [22]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig

config = AutoConfig.from_pretrained(
    checkpoint,
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

In [23]:
model = AutoModelForCausalLM.from_pretrained(checkpoint)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 124.4M parameters


# Data Collator

In [24]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [25]:
out = data_collator([tokenized_datasets[i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

input_ids shape: torch.Size([5, 500])
attention_mask shape: torch.Size([5, 500])
labels shape: torch.Size([5, 500])


In [26]:
samples = [tokenized_datasets[i] for i in range(8)]
batch = data_collator(samples)

# login into HuggingFace

In [27]:
from huggingface_hub import notebook_login

notebook_login()

# split tokenized dataset

In [27]:
# split the data into train and test set
#from sklearn.model_selection import train_test_split

#ds_train, ds_test = train_test_split(tokenized_datasets, test_size=0.2, random_state=42, shuffle=True)

In [28]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Detected operating system as Ubuntu/focal.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.0.9
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.


In [29]:
!sudo apt-get install git-lfs

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 74 not upgraded.
Need to get 7419 kB of archives.
After this operation, 16.0 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu focal/main amd64 git-lfs amd64 3.3.0 [7419 kB]
Fetched 7419 kB in 1s (11.4 MB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 69943 files and directories currently installed.)
Preparing to unpack .../git-lfs_3.3.0_amd64.deb ...
Unpacking git-lfs (3.3.0) ...
Setting up git-lfs (3.

# Training Aruguments

In [31]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="hiphop-ds-v3",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=5_000,
    logging_steps=5_000,
    gradient_accumulation_steps=32,
    num_train_epochs=10,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
    fp16=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets,
    #eval_dataset=ds_test,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


TypeError: __init__() got an unexpected keyword argument 'private'

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1735
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 32
  Total optimization steps = 60


Step,Training Loss,Validation Loss


In [ ]:
trainer.push_to_hub()